In [15]:
setwd ("/Users/ruyazhang/downloads")

In [16]:
dat1 <- read.csv("MedicareAllTelehealth.csv")

In [10]:
vars <- c("State", "Gender", "Credentials", "City", "ProviderType")

In [11]:
medicare1 <- dat1[,vars]

In [12]:
summary(medicare1, maxsum = 100)

 State      Gender                  Credentials                   City       
 AE:   44    :    17   MD                 :339781   NEW YORK        : 11651  
 AK: 1959   F:391309   M.D.               :339013   CHICAGO         :  9328  
 AL:15312   M:657249   D.O.               : 40297   HOUSTON         :  9256  
 AP:   11                                 : 34868   PHILADELPHIA    :  7955  
 AR: 9617              DO                 : 29543   BOSTON          :  7541  
 AZ:20759              PA-C               : 29222   ROCHESTER       :  6569  
 CA:78966              NP                 : 17554   BALTIMORE       :  6389  
 CO:16237              DPM                : 15396   COLUMBUS        :  6158  
 CT:15793              PA                 : 12488   SPRINGFIELD     :  6008  
 DC: 3100              M.D                : 11953   BROOKLYN        :  5928  
 DE: 4049              FNP                : 11150   LOS ANGELES     :  5803  
 FL:73622              O.D.               : 10975   DALLAS      

In [17]:
aggdata <- aggregate(NumberServices ~ State, dat1, sum)

In [18]:
aggdata[order(aggdata$NumberServices),]

,State,NumberServices
4,AP,589.0
1,AE,4460.0
58,XX,4492.0
29,MP,4804.0
59,ZZ,5811.0
14,GU,19324.0
52,VI,25649.0
2,AK,194642.0
57,WY,216437.5
44,PR,223525.0
